<a href="https://colab.research.google.com/github/thejsw/Quant/blob/master/PythonPortfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 파이썬과 재무 기초 지식

## 1.2 현금흐름, 이자율과 시간 가치

이자율은 화폐를 보유하는 경우에 생기는 기회비용  
단리: 원금 x (1 + 이율 x 기간)  
복리: 원금 x (1 + 이율) ** 기간

In [2]:
import math

In [7]:
a = 1
r = 1.0

n = 4
c_compound = a * (1+r / n) ** n

c_compound

2.44140625

## 1.3 NPV와 IRR

NPV(순현재가치): 미래에 발생할 현금흐름의 현재가치  
IRR(내부수익률):

In [9]:
# 현금흐름을 cashflows에 저장, i는 횟수, r은 이자율
cashflows = [12000, 15000, 18000, 21000, 26000]
i = 0
r = 0.015

# 최초 투자금액, 현금 유출이므로 (-)
npv = -70000

# cashflows 리스트를 반복해
for c in cashflows:
  i = i+1
  npv = npv + c/(1+r) ** i

npv


17516.929512135568

In [11]:
import scipy as sp

In [13]:

cashflows = [-70000, 12000, 15000, 18000, 21000, 26000]
r = 0.015

npv = 0
npv = sp.npv(r, cashflows)
npv

AttributeError: Module 'scipy' has no attribute 'npv'